In [1]:
import itertools
import json
import pandas as pd
import subprocess
import os

In [2]:
models = ["vit", "convnext", "xception"]
versions = ["1.4", "2.0", "2.1"]
squeezed = True

d = {}
for model, version in itertools.product(models, versions):
    file = f"{model}_{version}_squeezed_robustness.json"
    with open(file) as f:
        data = json.load(f)
        d[model + "_" + version] = data

    if squeezed:
        file = f"{model}_{version}_squeezed_square_partial_robustness.json"

        full_file = f"{model}_{version}_squeezed_square_robustness.json"
        if os.path.exists(full_file):
            file = full_file

        with open(file) as f:
            data = json.load(f)
            for key in data.keys():
                d[model + "_" + version][key] = data[key]

df = pd.DataFrame(d)


In [3]:
new_cols = [(x.split('_')[1], x.split('_')[0]) for x in df.columns]
df.columns = pd.MultiIndex.from_tuples(new_cols)
df = df.filter(regex='cm', axis=0)
df.index = df.index.str.replace('_cm', '')
df = df.rename(index=lambda x: x.replace("linf_0.01568627450980392", "$\epsilon=4/255$"))
df = df.rename(index=lambda x: x.replace("linf_0.03137254901960784", "$\epsilon=8/255$"))
df = df.rename(index=lambda x: x.replace("pad30", "pad"))
df = df.drop(["pad3", "pad10", "pad50", "colorjitter"])
df = df.reindex(sorted(df.columns), axis=1)

nat_14 = df['1.4'].loc['1.4']
nat_20 = df['2.0'].loc['2.0']
nat_21 = df['2.1'].loc['2.1']

nat = pd.concat([nat_14, nat_20, nat_21], axis=0)
nat.index = df.columns
df.loc['natural samples'] = nat

df['1.4'].loc['1.4'] = ["-", "-", "-"]
df['2.0'].loc['2.0'] = ["-", "-", "-"]
df['2.1'].loc['2.1'] = ["-", "-", "-"]

df = df.iloc[[9, 3, 4,5,6,7,8,0,1,2]]

/var/folders/98/wjg9j1cx1gbf3ky__8dpb3br0000gn/T/ipykernel_49005/2660187950.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['1.4'].loc['1.4'] = ["-", "-", "-"]
/var/folders/98/wjg9j1cx1gbf3ky__8dpb3br0000gn/T/ipykernel_49005/2660187950.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['2.0'].loc['2.0'] = ["-", "-", "-"]
/var/folders/98/wjg9j1cx1gbf3ky__8dpb3br0000gn/T/ipykernel_49005/2660187950.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [4]:
tex = """
\\begin{table}[]
\\caption{Confusion matrix for feature squeezing defense.}
\\resizebox{\\textwidth}{!}{%
\\begin{tabular}{l|cccccc|cccccc|cccccc}
Training set &
  \multicolumn{6}{l|}{\\textbf{1.4}} &
  \multicolumn{6}{l|}{\\textbf{2.0}} &
  \multicolumn{6}{l}{\\textbf{2.1}} \\\\
Model architecture &
  \multicolumn{2}{l}{ConvNext} &
  \multicolumn{2}{l}{ViT} &
  \multicolumn{2}{l|}{Xception} &
  \multicolumn{2}{l}{ConvNext} &
  \multicolumn{2}{l}{ViT} &
  \multicolumn{2}{l|}{Xception} &
  \multicolumn{2}{l}{ConvNext} &
  \multicolumn{2}{l}{ViT} &
  \multicolumn{2}{l}{Xception} \\\\ \\hline
"""

def get_color(i,k):
    if k%2 == 0:
        if i % 2 == 0:
            return "FFFFFF"
        else:
            return "EFEFEF"
    else:
        if i % 2 == 0:
            return "EFEFEF"
        else:
            return "FFFFFF"

for i, row in enumerate(df.iterrows()):
    if row[0] == "natural samples":
        tex += "\\multirow{2}{*}{\\textbf{Natural samples}} & \n"
    elif row[0] == "pad":
        tex += "\\textbf{NSP} & \n"
        for _ in range(17):
            tex += " & \n"
        tex += "\\\\ \n"
    elif row[0] == "$\epsilon=4/255$":
        tex += "\\textbf{A-PGD} ($L_\infty$) & \n"
        for _ in range(17):
            tex += " & \n"
        tex += "\\\\ \n"
    elif row[0] == "1.4":
        tex += "\\textbf{Other datasets} & \n"
        for _ in range(17):
            tex += " & \n"
        tex += "\\\\ \n"

    if row[0] != "natural samples":
        tex += "\\multirow{2}{*}{" + row[0] + "} & \n"

    for k, ((training_version, architecture), cf) in enumerate(row[1].iteritems()):
        color = get_color(i,k)
        if cf == "-":
            tex += " \\cellcolor[HTML]{" + color + "} - &"
            tex += " \\cellcolor[HTML]{" + color + "} - &"
        else:
            tex += " \\cellcolor[HTML]{" + color + "} " + str(cf[0][0]) + " &"
            tex += " \\cellcolor[HTML]{" + color + "} " + str(cf[0][1]) + " &"
    if tex[-1] == "&":
        tex = tex[:-1]
    tex += "\\\\ \n"
    tex += "& \n"
    for k, ((training_version, architecture), cf) in enumerate(row[1].iteritems()):
        color = get_color(i,k)
        if cf == "-":
            tex += " \\cellcolor[HTML]{" + color + "} - &"
            tex += " \\cellcolor[HTML]{" + color + "} - &"
        else:
            tex += " \\cellcolor[HTML]{" + color + "} " + str(cf[1][0]) + " &"
            tex += " \\cellcolor[HTML]{" + color + "} " + str(cf[1][1]) + " &"
    if tex[-1] == "&":
        tex = tex[:-1]
    tex += "\\\\ \n"

    


tex += """
\\end{tabular}%
}
\\end{table}
"""

subprocess.run("pbcopy", text=True, input=tex)

/var/folders/98/wjg9j1cx1gbf3ky__8dpb3br0000gn/T/ipykernel_49005/3018746334.py:56: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for k, ((training_version, architecture), cf) in enumerate(row[1].iteritems()):
/var/folders/98/wjg9j1cx1gbf3ky__8dpb3br0000gn/T/ipykernel_49005/3018746334.py:68: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for k, ((training_version, architecture), cf) in enumerate(row[1].iteritems()):


CompletedProcess(args='pbcopy', returncode=0)